## Exploratory Data Analysis Part 2

In [118]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [119]:
# Read from extracted data 
df_flights_raw = pd.read_csv("../data/raw/flights_raw.csv")
df_passengers_raw = pd.read_csv("../data/raw/passengers_raw.csv")
df_fuel_raw = pd.read_csv("../data/raw/fuel_raw.csv")

In [120]:
# Take copy to measure data loss after clean-up
df_flights = df_flights_raw.copy()
df_passengers = df_passengers_raw.copy()
df_fuel = df_fuel_raw.copy()

#### **Task 6**: How many states cover 50% of US air traffic? 

In [121]:
origin_dest = df_flights.filter(['flights', 'origin_city_name', 'dest_city_name'])
origin_dest[['origin_city', 'origin_state']] = origin_dest['origin_city_name'].str.split(", ",expand=True,) 
origin_dest[['dest_city', 'dest_state']] = origin_dest['dest_city_name'].str.split(", ",expand=True,) 

# drop unwanted column
origin_dest = origin_dest.drop(['origin_city_name', 'dest_city_name', 'origin_city', 'dest_city'], axis=1) 
origin_dest

,flights,origin_state,dest_state
0,1.0,NJ,DC
1,1.0,FL,MI
2,1.0,AL,GA
3,1.0,WA,CO
4,1.0,HI,HI
...,...,...,...
9995,1.0,VA,PA
9996,1.0,GA,GA
9997,1.0,UT,CO
9998,1.0,NC,NY


In [122]:
# Count of origin states
origin_state_count = origin_dest.groupby(by = 'origin_state').count().sort_values(by = 'flights', ascending=False)
origin_state_count.head()

,flights,dest_state
origin_state,,
CA,1157,1157
TX,992,992
FL,733,733
IL,662,662
NY,534,534


In [123]:
# Delete unwanted column
del origin_state_count['dest_state']

# Rename column name
origin_state_count.rename(columns={'flights':'origin_flights'}, inplace=True)
origin_state_count.head()

,origin_flights
origin_state,
CA,1157
TX,992
FL,733
IL,662
NY,534


In [124]:
# Count of dest states
dest_state_count = origin_dest.groupby(by = 'dest_state').count().sort_values(by = 'flights', ascending=False)

# Delete unwanted column
del dest_state_count['origin_state']

#rename column name
dest_state_count.rename(columns={'flights':'dest_flights'}, inplace=True)
dest_state_count.head()

,dest_flights
dest_state,
CA,1091
TX,973
FL,754
IL,599
NY,532


In [125]:
# Join dataframes on common states index
df_state = origin_state_count.join(dest_state_count)

# Fill nan value with zero
df_state['origin_flights'] = df_state['origin_flights'].fillna(0)
df_state['dest_flights'] = df_state['dest_flights'].fillna(0)

# Sum origin and dest
df_state['total_flights'] = df_state['origin_flights'] + df_state['dest_flights']

# Sort in descending order
df_state.sort_values('total_flights', ascending = False)
df_state.head()


,origin_flights,dest_flights,total_flights
origin_state,,,
CA,1157,1091,2248
TX,992,973,1965
FL,733,754,1487
IL,662,599,1261
NY,534,532,1066


In [126]:
df_state['percentage_origin_flights'] = df_state['origin_flights']/sum(df_state['origin_flights'])*100
df_state.reset_index(inplace=True)
df_state.head()

,origin_state,origin_flights,dest_flights,total_flights,percentage_origin_flights
0,CA,1157,1091,2248,11.57
1,TX,992,973,1965,9.92
2,FL,733,754,1487,7.33
3,IL,662,599,1261,6.62
4,NY,534,532,1066,5.34


In [127]:
df_state['percentage_dest_flights'] = df_state['dest_flights']/sum(df_state['dest_flights'])*100
df_state.reset_index(inplace=True)
df_state.head()

,index,origin_state,origin_flights,dest_flights,total_flights,percentage_origin_flights,percentage_dest_flights
0,0,CA,1157,1091,2248,11.57,10.91
1,1,TX,992,973,1965,9.92,9.73
2,2,FL,733,754,1487,7.33,7.54
3,3,IL,662,599,1261,6.62,5.99
4,4,NY,534,532,1066,5.34,5.32


In [128]:
df_state['percentage_total_flights'] = df_state['total_flights']/sum(df_state['total_flights'])*100
df_state.reset_index(inplace=True)
df_state.head()

,level_0,index,origin_state,origin_flights,dest_flights,total_flights,percentage_origin_flights,percentage_dest_flights,percentage_total_flights
0,0,0,CA,1157,1091,2248,11.57,10.91,11.240
1,1,1,TX,992,973,1965,9.92,9.73,9.825
2,2,2,FL,733,754,1487,7.33,7.54,7.435
3,3,3,IL,662,599,1261,6.62,5.99,6.305
4,4,4,NY,534,532,1066,5.34,5.32,5.330


In [129]:
top50_origin = []
percentage_origin = 0

for index, row in df_state.iterrows():
    percentage_origin += row['percentage_origin_flights']
    if percentage_origin > 50:
        break
    top50_origin.append(row['origin_state'])
    
    
print('Number of US States comprising 50% US origin air flights: ', len(top50_origin))
print('US States: ', top50_origin)

Number of US States comprising 50% US origin air flights:  6
US States:  ['CA', 'TX', 'FL', 'IL', 'NY', 'GA']


In [130]:
top50_dest = []
percentage_dest = 0

for index, row in df_state.iterrows():
    percentage_dest += row['percentage_dest_flights']
    if percentage_dest > 50:
        break
    top50_dest.append(row['origin_state'])
    
    
print('Number of US States comprising 50% US destination air flights: ', len(top50_dest))
print('US States: ', top50_dest)

Number of US States comprising 50% US destination air flights:  7
US States:  ['CA', 'TX', 'FL', 'IL', 'NY', 'GA', 'NC']


#### **Task 7**: Test the hypothesis whether planes fly faster when there is the departure delay? 

#### **Task 8**: When (which hour) do most 'LONG', 'SHORT', 'MEDIUM' haul flights take off?

#### **Task 9**: Find the top 10 the bussiest airports. Does the biggest number of flights mean that the biggest number of passengers went through the particular airport? How much traffic do these 10 airports cover?

#### **Task 10**: Do bigger delays lead to bigger fuel comsumption per passenger? 
We need to do four things to answer this as accurate as possible:
- Find out average monthly delay per air carrier (monthly delay is sum of all delays in 1 month)
- Find out distance covered monthly by different air carriers
- Find out number of passengers that were carried by different air carriers
- Find out total fuel comsumption per air carrier.

Use this information to get the average fuel comsumption per passenger per km. Is this higher for the airlines with bigger average delays?